### This kernel is forked from https://www.kaggle.com/bminixhofer/aggregated-features-lightgbm . I have done some changes to get better score 

In [1]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
#from matplotlib_venn import venn2, venn2_circles
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import scipy
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold
import time

sns.set()
%matplotlib inline

NFOLDS = 5
SEED = 42

/home/g492652607/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data Loading


In [2]:
train = pd.read_csv('/home/g492652607/data/train.csv')
test = pd.read_csv('/home/g492652607/data/test.csv')
gp = pd.read_csv('/home/g492652607/data/aggregated_features.csv') 
train = train.merge(gp, on='user_id', how='left')
test = test.merge(gp, on='user_id', how='left')

ntrain = train.shape[0]
ntest = test.shape[0]
agg_cols = list(gp.columns)[1:]

del gp
gc.collect()

train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,avg_days_up_user,avg_times_up_user,n_user_items
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),...,400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789,8.000000,2.000000,2
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,...,3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000,NaN,NaN,1
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,...,4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177,4.428571,1.142857,9
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,...,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323,16.714286,2.642857,32
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",...,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797,NaN,NaN,1


# Feature Engineering
### Text cleaning does not help So, I am commenting them

In [3]:
blend = pd.read_csv('/home/g492652607/data/blend.csv') 
train=train.merge(blend, on='item_id', how='left')
test=test.merge(blend, on='item_id', how='left')
idx=blend['item_id']

img_t=pd.read_csv('/home/g492652607/data/test_0.csv')
img0=pd.read_csv('/home/g492652607/data/train-0.csv')
img1=pd.read_csv('/home/g492652607/data/train-1.csv')
img2=pd.read_csv('/home/g492652607/data/train-2.csv')
img4=pd.read_csv('/home/g492652607/data/train-4.csv')
img3=pd.read_csv('/home/g492652607/data/train-3.csv')
img=pd.concat([img_t,img0,img1,img2,img3,img4],axis=0)
img['image']=list(img['image'].str.split('.').str[0])
train=train.merge(img, on='image', how='left')
test=test.merge(img, on='image', how='left')

img.drop('image',axis=1,inplace=True)
img_col=list(img.columns)

vgg1 = pd.read_csv('/home/g492652607/data/VGG16_1.csv')
vgg2 = pd.read_csv('/home/g492652607/data/VGG16_2.csv')
vgg3 = pd.read_csv('/home/g492652607/data/VGG16_3.csv')
vgg=pd.merge(vgg1,vgg2,on='item_id',how='left')
vgg=vgg.merge(vgg3,on='item_id',how='left')
train=train.merge(vgg, on='item_id', how='left')
test=test.merge(vgg, on='item_id', how='left')
vgg.drop('item_id',axis=1,inplace=True)
img_col2=list(vgg.columns)

In [4]:
print(train.columns)

Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability',
       'avg_days_up_user', 'avg_times_up_user', 'n_user_items', 'rnn_pred',
       't', 'd', 'ridge_pred', 'cb_pred', 'user_price_aver', 'user_price_std',
       'user_seq_aver', 'user_seq_std',
       'user_id_parent_category_name_category_name_count',
       'user_id_parent_category_name_category_name_aver_pri',
       'region_city_parent_category_name_category_name_count', 'dullness',
       'whiteness', 'average_pixel_width', 'dominant_red', 'dominant_green',
       'dominant_blue', 'average_red', 'average_green', 'average_blue',
       'image_size', 'width', 'height', 'blurrness', 'im_tsvd_0', 'im_tsvd_1',
       'im_tsvd_2', 'im_tsvd_3', 'im_tsvd_4', 'im_tsvd_5', 'im_tsvd_6',
       'im_tsvd_7', 'im_tsvd_8', 'im_ts

In [5]:
for df in [train, test]:
    df['description'].fillna('unknowndesc', inplace=True)
    df['title'].fillna('unknowntitle', inplace=True)

    df['weekday'] = pd.to_datetime(df['activation_date']).dt.weekday
 #   df['dayofmonth'] = pd.to_datetime(df['activation_date']).dt.day
    
    for col in ['description', 'title']:
        df['num_words_' + col] = df[col].apply(lambda comment: len(comment.split()))
        df['num_unique_words_' + col] = df[col].apply(lambda comment: len(set(w for w in comment.split())))

    df['words_vs_unique_title'] = df['num_unique_words_title'] / df['num_words_title'] * 100
    df['words_vs_unique_description'] = df['num_unique_words_description'] / df['num_words_description'] * 100
    
    df['city'] = df['region'] + '_' + df['city']
    df['num_desc_punct'] = df['description'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    
    for col in agg_cols:
        df[col].fillna(-1, inplace=True)
        
    for col in img_col:
        df[col].fillna(-1, inplace=True)

In [6]:
count_vectorizer_title = CountVectorizer(stop_words=stopwords.words('russian'), lowercase=True, min_df=2,max_features=5000)

title_counts = count_vectorizer_title.fit_transform(train['title'].append(test['title']))

train_title_counts = title_counts[:len(train)]
test_title_counts = title_counts[len(train):]


count_vectorizer_desc = TfidfVectorizer(stop_words=stopwords.words('russian'), 
                                        lowercase=True, ngram_range=(1, 2),
                                        max_features=5000)

desc_counts = count_vectorizer_desc.fit_transform(train['description'].append(test['description']))

train_desc_counts = desc_counts[:len(train)]
test_desc_counts = desc_counts[len(train):]

train_title_counts.shape, train_desc_counts.shape


((1503424, 5000), (1503424, 5000))

In [7]:
target = 'deal_probability'
predictors = [
    'num_desc_punct',
    'words_vs_unique_description', 'num_unique_words_description', 'num_unique_words_title', 'num_words_description', 'num_words_title',
    'avg_times_up_user', 'avg_days_up_user', 'n_user_items', 
    'price', 'item_seq_number', 'user_price_aver', 'user_price_std', 'user_seq_aver', 'user_seq_std',
    'user_id_parent_category_name_category_name_count', 
    'user_id_parent_category_name_category_name_aver_pri',
    'region_city_parent_category_name_category_name_count','rnn_pred','t','d','ridge_pred','cb_pred'
]
categorical = [
    'image_top_1', 'param_1', 'param_2', 'param_3', 
    'city', 'region', 'category_name', 'parent_category_name', 'user_type'
]

predictors = predictors + categorical+img_col+img_col2

In [8]:
for feature in categorical:
    print(f'Transforming {feature}...')
    encoder = LabelEncoder()
    train[feature].fillna('unknown',inplace=True)
    test[feature].fillna('unknown',inplace=True)
    encoder.fit(train[feature].append(test[feature]).astype(str))
    
    train[feature] = encoder.transform(train[feature].astype(str))
    test[feature] = encoder.transform(test[feature].astype(str))

Transforming image_top_1...
Transforming param_1...
Transforming param_2...
Transforming param_3...
Transforming city...
Transforming region...
Transforming category_name...
Transforming parent_category_name...
Transforming user_type...


In [9]:
train["price"] = np.log(train["price"]+0.001)
train["price"].fillna(-999,inplace=True)
train["image_top_1"].fillna(-999,inplace=True)

test["price"] = np.log(test["price"]+0.001)
test["price"].fillna(-999,inplace=True)
test["image_top_1"].fillna(-999,inplace=True)


# LightGBM 


In [10]:
feature_names = np.hstack([
    count_vectorizer_desc.get_feature_names(),
    count_vectorizer_title.get_feature_names(),
    predictors
])
print('Number of features:', len(feature_names))

x_test = scipy.sparse.hstack([
    test_desc_counts,
    test_title_counts,
    test.loc[:, predictors]
], format='csr')

X = scipy.sparse.hstack([
    train_desc_counts,
    train_title_counts,
    train.loc[: , predictors]
], format='csr')
y = train.deal_probability

Number of features: 10082


In [ ]:
def cross_validate_lgb(params, x_train, y_train, x_test, kf, cat_cols=[],
                       verbose=True, verbose_eval=50, use_cat=True, use_rank=False):
    start_time = time.time()
    train_pred = np.zeros((ntrain))
    test_pred = np.zeros((ntest))

    if len(cat_cols)==0: use_cat=False

    # use the k-fold object to enumerate indexes for each training and validation fold
    for i, (train_index, val_index) in enumerate(kf): # folds 1, 2 ,3 ,4, 5
        # example: training from 1,2,3,4; validation from 5
        print('\nFold {}'.format(i))
        x_train_kf, x_val_kf = x_train[train_index], x_train[val_index]
        y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]

        if use_cat:
            lgb_train = lgb.Dataset(x_train_kf, y_train_kf, feature_name=list(feature_names),categorical_feature=cat_cols)
            lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train, feature_name=list(feature_names),categorical_feature=cat_cols)
        else:
            lgb_train = lgb.Dataset(x_train_kf, y_train_kf, feature_name=list(feature_names))
            lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train, feature_name=list(feature_names))

        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=4000,
                        valid_sets=lgb_val,
                        early_stopping_rounds=30,
                        verbose_eval=verbose_eval)

        val_pred = gbm.predict(x_val_kf)

        if use_rank:
            train_pred[val_index] += probability_to_rank(val_pred)
            test_pred += probability_to_rank(gbm.predict(x_test))
            # test_pred += gbm.predict(x_test)
        else:
            train_pred[val_index] += val_pred
            test_pred += gbm.predict(x_test)

        # test_pred += gbm.predict(x_test)
        rms = sqrt(mean_squared_error(y_val_kf.values, val_pred))
        if verbose:
            print('fold cv {} RMSE score is {:.6f}'.format(i, rms))

    test_pred /=NFOLDS

    cv_rms = sqrt(mean_squared_error(y_train, train_pred))
    if verbose:
        print('cv RMSE score is {:.6f}'.format(cv_rms))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
    return train_pred.reshape(-1, 1),test_pred.reshape(-1, 1)

In [ ]:
kf = KFold(ntrain, n_folds=NFOLDS, random_state=SEED)
lgb_params = {'learning_rate':0.02,
              'task':'train',
              'boosting_type':'gbdt',
              'metric':'rmse',
              'objective':'regression',
              'num_leaves':1000,
              'max_depth':18,
              'min_data_in_leaf':25, #Defaut 20
              'feature_fraction': 0.43,
              'feature_fraction_seed':0,
              'bagging_fraction': 0.46,
              #'bagging_freq': 2,
              'bagging_seed':0,
              'verbose':0,
              'num_threads':4 #Put to 4 if you are leaving computer
              }
gbm_oof_train, gbm_oof_test=cross_validate_lgb(lgb_params,X, y, x_test, kf, cat_cols=categorical, use_cat=True, 
                            verbose_eval=False, use_rank=False)


gbm_preds = np.concatenate([gbm_oof_train, gbm_oof_test])


Fold 0


/home/g492652607/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [13]:
gbm_oof_test

array([[0.40987939],
       [0.14368834],
       [0.1128581 ],
       ...,
       [0.06808092],
       [0.40200041],
       [0.08980047]])

In [14]:
sub = pd.DataFrame(gbm_preds,columns=['gbm4_pred'])
sub1 = pd.DataFrame(idx,columns=['item_id'])
sub1=sub1.set_index(sub.index)
gb=pd.concat([sub1,sub],axis=1)


In [15]:
gb.to_csv('gbm1000tree.csv', index=False)

In [17]:
subm = pd.read_csv('/home/g492652607/data/sample_submission.csv')
subm['deal_probability'] = np.clip(gbm_oof_test, 0, 1)
subm.to_csv('lgbtest1000.csv', index=False)